## Boiler Plate

In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [13]:
import pandas as pd
import numpy as np
from fastai.vision import *
from fastai.callbacks.hooks import *
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

In [14]:
path = Path('../data/celeba/faces/')

In [15]:
## Function to filter validation samples
def validation_func(x):
    return 'validation' in x
src = (ImageItemList.from_csv(path, csv_name='labels.csv')
       .split_by_valid_func(validation_func)
       .label_from_df(cols='tags',label_delim=' '))

In [16]:
tfms = get_transforms(do_flip=False, flip_vert=False, max_rotate=0, max_lighting=0, max_zoom=1, max_warp=0)
data = (src.transform(tfms=tfms,size=256)
       .databunch(bs=64).normalize(imagenet_stats))

In [17]:
print(data.c,'\n',data.classes)

40 
 ['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']


In [18]:
learn = create_cnn(data, models.resnet50, pretrained=False)
learn.load("ff_stage-1-256-rn50")

Learner(data=ImageDataBunch;

Train: LabelList
y: MultiCategoryList (158149 items)
[MultiCategory Arched_Eyebrows;Attractive;Brown_Hair;Heavy_Makeup;High_Cheekbones;Mouth_Slightly_Open;No_Beard;Pointy_Nose;Smiling;Straight_Hair;Wearing_Earrings;Wearing_Lipstick;Young, MultiCategory Bags_Under_Eyes;Big_Nose;Brown_Hair;High_Cheekbones;Mouth_Slightly_Open;No_Beard;Smiling;Young, MultiCategory Arched_Eyebrows;Attractive;Big_Lips;Heavy_Makeup;Narrow_Eyes;No_Beard;Pointy_Nose;Wearing_Lipstick;Young, MultiCategory Arched_Eyebrows;Attractive;Big_Lips;Brown_Hair;Heavy_Makeup;Mouth_Slightly_Open;No_Beard;Wavy_Hair;Wearing_Earrings;Wearing_Lipstick;Young, MultiCategory 5_o_Clock_Shadow;Attractive;Bags_Under_Eyes;Big_Lips;Big_Nose;Black_Hair;Bushy_Eyebrows;Male;No_Beard;Pointy_Nose;Straight_Hair;Young]...
Path: ../data/celeba/faces
x: ImageItemList (158149 items)
[Image (3, 186, 186), Image (3, 210, 200), Image (3, 218, 200), Image (3, 175, 175), Image (3, 195, 195)]...
Path: ../data/celeba/faces;

In [19]:
# this is a hook (learned about it here: https://forums.fast.ai/t/how-to-find-similar-images-based-on-final-embedding-layer/16903/13)
class SaveFeatures():
    features=None
    def __init__(self, m): 
        self.hook = m.register_forward_hook(self.hook_fn)
        self.features = None
    def hook_fn(self, module, input, output): 
        out = output.detach().cpu().numpy()
        if isinstance(self.features, type(None)):
            self.features = out
        else:
            self.features = np.row_stack((self.features, out))
    def remove(self): 
        self.hook.remove()
        
sf = SaveFeatures(learn.model[1][1])

In [20]:
_ = learn.get_preds(ds_type=DatasetType.Train)
_ = learn.get_preds(ds_type=DatasetType.Valid)

In [26]:
## Converting in a dictionary of path:featurevector
feature_dict = dict(zip(list(data.train_ds.items)+list(data.valid_ds.items),sf.features))

In [36]:
## Exporting as pickle
import pickle
pickle.dump(feature_dict, open(path/"feature_dict.p", "wb" ) )